# Shark Attack Cleaner

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('attacks.csv', sep=',', encoding='ANSI')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


#### Limpando colunas inuteis (nulas e com dados repetidos de outras colunas)

In [5]:
# ELIMINANDO AS COLUNAS NULAS OU COM INFO REPETID
df_fatal = df.loc[:, ['Date', 
                      'Year', 
                      'Type',
                      'Country', 
                      'Activity', 
                      'Injury', 
                      'Fatal (Y/N)', 
                      ]].rename(columns={'Fatal (Y/N)': 'Fatal',
                                                         'Sex ': 'Sex' })
df_fatal.head(1)

,Date,Year,Type,Country,Activity,Injury,Fatal
0,25-Jun-2018,2018.0,Boating,USA,Paddling,"No injury to occupant, outrigger canoe and pad...",N


##### Limpando linhas totalmente nulas

In [6]:

linenan = df_fatal.loc[df_fatal.isnull().all(axis=1)].index
df_fatal = df_fatal.drop(index=linenan)
df_fatal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6301
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      6302 non-null   object 
 1   Year      6300 non-null   float64
 2   Type      6298 non-null   object 
 3   Country   6252 non-null   object 
 4   Activity  5758 non-null   object 
 5   Injury    6274 non-null   object 
 6   Fatal     5763 non-null   object 
dtypes: float64(1), object(6)
memory usage: 393.9+ KB


## FATAL? 

### Limpando a coluna 'Fatal (Y/N))'

In [7]:
#obtendo os valores da coluna para tratamento e padronizacao
df_fatal.loc[:,'Fatal'].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [8]:
def limp_fatal(row):
    '''recebe as linhas e baseada dos dados da colunas:
                            Fatal e Injury, padroniza os dados com
                                                            0; 1; np.nan
                            se fatal = 0
                            se nao fatal = 1
                            sem o dado = np.nan
    '''

    if isinstance(row['Fatal'], float) or row['Fatal'] == 'UNKNOWN':
        if row['Injury'] == 'NaN':
            return np.nan
        elif bool(re.search('fatal', row['Injury'].lower())):
            return 1
        else:
            return 0
    elif 'n' == row['Fatal'].lower().strip() or row['Fatal'] == '2017' or row['Fatal'] == 'M':
        return 0
    elif 'y' == row['Fatal'].lower().strip():
        return 1

###### A coluna 'Injury' contem dados ausentes da coluna 'Fatal'
Padronizando os dados de 'Injury' para que nulos sejam strings

In [9]:
df_fatal.loc[df_fatal.loc[:, 'Injury'].isna(), 'Injury'] = 'NaN'

##### rodando a funcao de limpesa de 'Fatal'
limp_fatal()

In [10]:
df_fatal.loc[:, 'Fatal'] = df_fatal.loc[:].apply(limp_fatal, axis=1)

In [11]:
#serie fatal limpa
ind_fatal = df_fatal['Fatal']
ind_fatal.value_counts()


0.0    4834
1.0    1441
Name: Fatal, dtype: int64

In [12]:
df_fatal.loc[:, 'Country'] = df_fatal.loc[:, 'Country'].apply(lambda x : re.sub('[^\w /]', '', x.strip()) if isinstance(x, str) else np.nan)

In [13]:
fat_country = df_fatal.groupby('Country', as_index=False)[['Country', "Fatal"]].agg(('count', sum, 'mean')).sort_values(by=[('Fatal', 'count')] , ascending=False)
fat_country.head(50)

Fatal                 
                     count    sum      mean
Country                                    
USA                   2224  198.0  0.089029
AUSTRALIA             1332  295.0  0.221471
SOUTH AFRICA           578  114.0  0.197232
PAPUA NEW GUINEA       134   56.0  0.417910
NEW ZEALAND            126   25.0  0.198413
BRAZIL                 110   38.0  0.345455
BAHAMAS                109   10.0  0.091743
MEXICO                  90   44.0  0.488889
ITALY                   71   15.0  0.211268
FIJI                    62   22.0  0.354839
PHILIPPINES             62   37.0  0.596774
REUNION                 60   29.0  0.483333
NEW CALEDONIA           53   19.0  0.358491
CUBA                    45   25.0  0.555556
MOZAMBIQUE              44   19.0  0.431818
SPAIN                   44    8.0  0.181818
EGYPT                   40   12.0  0.300000
INDIA                   38   21.0  0.552632
JAPAN                   34   12.0  0.352941
CROATIA                 34   16.0  0.470588
PANAMA                  32   18.0  0.562500
SOLOMON ISLANDS         30   12.0  0.400000
IRAN                    29   13.0  0.448276
JAMAICA                 27   14.0  0.518519
FRENCH POLYNESIA        25    2.0  0.080000
GREECE                  25   15.0  0.600000
HONG KONG               24   17.0  0.708333
INDONESIA               23   15.0  0.652174
ENGLAND                 23    6.0  0.260870
PACIFIC OCEAN           19    8.0  0.421053
TONGA                   18    6.0  0.333333
COSTA RICA              17    8.0  0.470588
ATLANTIC OCEAN          17    8.0  0.470588
BERMUDA                 16    3.0  0.187500
VIETNAM                 15    2.0  0.133333
VANUATU                 14    9.0  0.642857
SRI LANKA               14    3.0  0.214286
MARSHALL ISLANDS        13    2.0  0.153846
TURKEY                  12    3.0  0.250000
FRANCE                  12    3.0  0.250000
CANADA                  12    3.0  0.250000
IRAQ                    12    3.0  0.250000
SOUTH ATLANTIC OCEAN    12    4.0  0.333333
UNITED KINGDOM          11    2.0  0.181818
VENEZUELA               11    5.0  0.454545
NEW GUINEA              10    6.0  0.600000
MAURITIUS               10    4.0  0.400000
KENYA                   10    6.0  0.600000
SENEGAL                 10    3.0  0.300000
TAIWAN                   9    3.0  0.333333

In [14]:
ano_0 = df_fatal.loc[:,'Year'] == 0
ano_na = df_fatal.loc[:,'Year'].isna()
df_fatal['Date']

0       25-Jun-2018
1       18-Jun-2018
2       09-Jun-2018
3       08-Jun-2018
4       04-Jun-2018
           ...     
6297    Before 1903
6298    Before 1903
6299      1900-1905
6300      1883-1889
6301      1845-1853
Name: Date, Length: 6302, dtype: object

In [15]:
ano = df_fatal.loc[:, 'Date'].str.extract('(\d{4})')
df_fatal.loc[:, 'Year' ] = ano[0].astype(float)
dropano = df_fatal['Year'] >= 1950
df_fatal = df_fatal[dropano]
df_fatal.loc[:, 'Year'] = df_fatal.loc[:, 'Year'].astype(int)
df_fatal


,Date,Year,Type,Country,Activity,Injury,Fatal
0,25-Jun-2018,2018,Boating,USA,Paddling,"No injury to occupant, outrigger canoe and pad...",0.0
1,18-Jun-2018,2018,Unprovoked,USA,Standing,Minor injury to left thigh,0.0
2,09-Jun-2018,2018,Invalid,USA,Surfing,Injury to left lower leg from surfboard skeg,0.0
3,08-Jun-2018,2018,Unprovoked,AUSTRALIA,Surfing,Minor injury to lower leg,0.0
4,04-Jun-2018,2018,Provoked,MEXICO,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,0.0
...,...,...,...,...,...,...,...
6254,Before 1957,1957,Provoked,CUBA,"Shark fishing, knocked overboard","FATAL, hip bitten PROVOKED INCIDENT",1.0
6255,Before 1956,1956,Unprovoked,MARSHALL ISLANDS,Swimming,Buttocks bitten,0.0
6256,Before 1956,1956,Unprovoked,KIRIBATI,Diving,No injury,0.0
6257,Before Mar-1956,1956,Unprovoked,NORTH PACIFIC OCEAN,"Fishing, wading with string of fish",Survived,0.0


In [16]:
df_fatal.loc[:, 'Type'].unique()
df_fatal.loc[(df_fatal['Type'] == 'Boat') | (df_fatal['Type'] == 'Boatomg'), 'Type'] = 'Boating'
df_fatal.loc[df_fatal.loc[:, 'Type'].isna(), 'Type'] = 'Questionable'


In [17]:
df_fatal.loc[df_fatal.loc[:, 'Fatal'].isna(), :]

,Date,Year,Type,Country,Activity,Injury,Fatal
1270,20-Apr-2008,2008,Unprovoked,AUSTRALIA,NaN,NaN,NaN
1366,Jul-2007,2007,Invalid,SENEGAL,Murder,NaN,NaN
2250,07-Jun-1997,1997,Unprovoked,BRAZIL,Bathing,NaN,NaN
2267,29-Dec-1996,1996,Unprovoked,AUSTRALIA,Surfing,NaN,NaN
2648,1990,1990,Unprovoked,USA,Surfing,NaN,NaN
3038,Mar-1981,1981,Unprovoked,BRAZIL,Diving,NaN,NaN
3207,04-Oct-1975,1975,Invalid,USA,Surfing,NaN,NaN
3298,27-Aug-1973,1973,Unprovoked,AUSTRALIA,NaN,NaN,NaN
3434,22-May-1969,1969,Unprovoked,DOMINICAN REPUBLIC,Surfing,NaN,NaN
3457,22-Sep-1968,1968,Invalid,USA,Surfing,NaN,NaN


In [18]:
df_fatal.loc[df_fatal.loc[:, 'Injury'] == 'NaN', 'Injury'] = np.nan
df_fatal = df_fatal.loc[df_fatal['Injury'] == df_fatal['Injury'], :]
df_fatal.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4543 entries, 0 to 6258
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      4543 non-null   object 
 1   Year      4543 non-null   int32  
 2   Type      4543 non-null   object 
 3   Country   4527 non-null   object 
 4   Activity  4218 non-null   object 
 5   Injury    4543 non-null   object 
 6   Fatal     4543 non-null   float64
dtypes: float64(1), int32(1), object(5)
memory usage: 266.2+ KB


In [19]:
# ['Boating', 'Unprovoked', 'Invalid', 'Provoked',
# 'Questionable', 'Sea Disaster', nan, 'Boat', 'Boatomg']
df_fatal.loc[(df_fatal['Type'] == 'Boat') | (df_fatal['Type'] == 'Boatomg'), 'Type'] = 'Boating'
df_fatal.loc[df_fatal.loc[:, 'Type'].isna(), 'Type'] = 'Questionable'

# excluindo mais algumas linhas sem dados relevantes
df_fatal.loc[df_fatal.loc[:, 'Injury'] == 'NaN', 'Injury'] = np.nan
df_fatal = df_fatal.loc[df_fatal['Injury'] == df_fatal['Injury'], :]


ataques = (df_fatal['Type'] == 'Provoked') | (df_fatal['Type'] == 'Unprovoked')
df_fatal.loc[(df_fatal['Fatal'] == 0) & ataques, 'NOT Fatal'] = 1
df_fatal.loc[df_fatal['NOT Fatal'].isna(), 'NOT Fatal'] = 0
df_fatal.loc[:,'NOT Fatal'] = df_fatal['NOT Fatal'].astype(int)

mortsematk = (df_fatal['Fatal'] == 1) & (df_fatal['Type'] == 'Invalid')
df_fatal.loc[mortsematk, 'Fatal'] = 0
df_fatal['Fatal'] = df_fatal['Fatal'].astype(int)

inva_boat = pd.get_dummies(df_fatal['Type']).iloc[:, :2]


C:\Users\eandr\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [20]:
df_fatal= pd.concat([df_fatal, inva_boat], axis=1)
df_fatal = df_fatal.drop(columns='Date').reset_index()
df_fatal = df_fatal.drop(index=4063)
df_fatal

,index,Year,Type,Country,Activity,Injury,Fatal,NOT Fatal,Boating,Invalid
0,0,2018,Boating,USA,Paddling,"No injury to occupant, outrigger canoe and pad...",0,0,1,0
1,1,2018,Unprovoked,USA,Standing,Minor injury to left thigh,0,1,0,0
2,2,2018,Invalid,USA,Surfing,Injury to left lower leg from surfboard skeg,0,0,0,1
3,3,2018,Unprovoked,AUSTRALIA,Surfing,Minor injury to lower leg,0,1,0,0
4,4,2018,Provoked,MEXICO,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
4538,6254,1957,Provoked,CUBA,"Shark fishing, knocked overboard","FATAL, hip bitten PROVOKED INCIDENT",1,0,0,0
4539,6255,1956,Unprovoked,MARSHALL ISLANDS,Swimming,Buttocks bitten,0,1,0,0
4540,6256,1956,Unprovoked,KIRIBATI,Diving,No injury,0,1,0,0
4541,6257,1956,Unprovoked,NORTH PACIFIC OCEAN,"Fishing, wading with string of fish",Survived,0,1,0,0


In [21]:
perlocal = df_fatal.groupby('Country')[['Fatal', 'NOT Fatal', 'Boating', 'Invalid']].sum()
perano = df_fatal.groupby('Year')[['Fatal', 'NOT Fatal', 'Boating', 'Invalid']].sum()

In [22]:
perano

,Fatal,NOT Fatal,Boating,Invalid
Year,,,,
1950,10,26,4,4
1951,12,18,0,3
1952,10,16,2,2
1953,6,24,3,1
1954,15,22,2,3
...,...,...,...,...
2014,7,99,11,9
2015,9,108,12,13
2016,6,109,7,9


In [23]:
perlocal
df_fatal 

,index,Year,Type,Country,Activity,Injury,Fatal,NOT Fatal,Boating,Invalid
0,0,2018,Boating,USA,Paddling,"No injury to occupant, outrigger canoe and pad...",0,0,1,0
1,1,2018,Unprovoked,USA,Standing,Minor injury to left thigh,0,1,0,0
2,2,2018,Invalid,USA,Surfing,Injury to left lower leg from surfboard skeg,0,0,0,1
3,3,2018,Unprovoked,AUSTRALIA,Surfing,Minor injury to lower leg,0,1,0,0
4,4,2018,Provoked,MEXICO,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
4538,6254,1957,Provoked,CUBA,"Shark fishing, knocked overboard","FATAL, hip bitten PROVOKED INCIDENT",1,0,0,0
4539,6255,1956,Unprovoked,MARSHALL ISLANDS,Swimming,Buttocks bitten,0,1,0,0
4540,6256,1956,Unprovoked,KIRIBATI,Diving,No injury,0,1,0,0
4541,6257,1956,Unprovoked,NORTH PACIFIC OCEAN,"Fishing, wading with string of fish",Survived,0,1,0,0


In [24]:
df_fatal.loc[:,[ 'Fatal', 'NOT Fatal', 'Boating', 'Invalid']].agg(('mean','sum')).T 


,mean,sum
Fatal,0.141788,644.0
NOT Fatal,0.715103,3248.0
Boating,0.053060,241.0
Invalid,0.082783,376.0


In [25]:
f = (df_fatal['Type'] == 'Provoked') | (df_fatal['Type'] == 'Unprovoked')
df_fatal.loc[f,[ 'Fatal', 'NOT Fatal', 'Boating', 'Invalid']].agg(('mean','sum')).T 


,mean,sum
Fatal,0.150628,576.0
NOT Fatal,0.849372,3248.0
Boating,0.000000,0.0
Invalid,0.000000,0.0


In [26]:
df_fatal.loc[df_fatal['Type'] == 'Provoked',[ 'Fatal', 'NOT Fatal', 'Boating', 'Invalid']].agg(('mean','sum')).T 

,mean,sum
Fatal,0.027586,12.0
NOT Fatal,0.972414,423.0
Boating,0.000000,0.0
Invalid,0.000000,0.0


In [27]:
df_fatal.loc[df_fatal['Type'] == 'Unprovoked',[ 'Fatal', 'NOT Fatal', 'Boating', 'Invalid']].agg(('mean','sum')).T 

,mean,sum
Fatal,0.166421,564.0
NOT Fatal,0.833579,2825.0
Boating,0.000000,0.0
Invalid,0.000000,0.0


In [28]:
df.loc[df['Type'] == 'Unprovoked', 'Type'].count()

4595

In [29]:
df.loc[df['Type'] == 'Provoked', 'Type'].count()

574

In [39]:
df_fatal.groupby('Year').mean().sort_values('Fatal')


,index,Fatal,NOT Fatal,Boating,Invalid
Year,,,,,
1969,3526.344828,0.000000,0.931034,0.000000,0.068966
2018,26.000000,0.018868,0.811321,0.018868,0.113208
2007,1355.781818,0.045455,0.845455,0.027273,0.081818
2016,299.755725,0.045802,0.832061,0.053435,0.068702
2001,1935.500000,0.054348,0.782609,0.076087,0.086957
...,...,...,...,...,...
1973,3298.000000,0.346154,0.653846,0.000000,0.000000
1958,4363.100000,0.350000,0.483333,0.066667,0.083333
1954,4321.476190,0.357143,0.523810,0.047619,0.071429
